### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import os

# File to Load (Remember to Change These)
school_data_to_load = os.path.join("Resources", "schools_complete.csv")
student_data_to_load = os.path.join("Resources", "students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school = pd.read_csv(school_data_to_load)
student = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
df = pd.merge(student, school, how="left", on=["school_name", "school_name"])
df

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Count the total number of schools
Total_Schools = df['school_name'].nunique()

#Count the total number of students
Total_Students = df['student_name'].count()

# Find the total budget for all schools in the district
Total_Budget = school['budget'].sum()

# Find the average Scores of Math and reading rounded to the 6th place as per example
Avg_Math = (df['math_score'].mean()).round(6)
Avg_Read = (df['reading_score'].mean()).round(6)

# Find total number of students who passed Math
Pass_Math = sum(x >= 70 for x in df['math_score'])

# Find total number of students who passed Reading
Pass_Read = sum(x >= 70 for x in df['reading_score'])

#Find total number of students who passed Math and Reading
Pass_Read_Math = len(df.loc[(df['math_score'] >= 70) & (df['reading_score'] >= 70)])

# Calculate all passing percentages
Pass_Math_Per = ((Pass_Math / Total_Students) * 100)
Pass_Read_Per = ((Pass_Read / Total_Students) * 100)
Pass_Read_Math_Per = ((Pass_Read_Math / Total_Students) * 100)


In [3]:
# Create a Summary Data Frame that includes all of the above calculated data
District_df = pd.DataFrame([{'Total Schools': Total_Schools, 'Total Students': Total_Students, 'Total Budget': Total_Budget, 
    'Average Math Score': Avg_Math, 'Average Reading Score' : Avg_Read, 'Pass Math (%)' : Pass_Math_Per, 
    'Pass Reading (%)' : Pass_Read_Per, 'Pass Overall (%)': Pass_Read_Math_Per}])

# Format columns of Summary Data Frame for ease of reading
District_df.style.format({"Total Students": '{:,}', 'Total Budget': '${:,}', 'Average Math Score': '{:.2f}',
                        'Average Reading Score': '{:.2f}', 'Pass Math (%)': '{:.2f}', 'Pass Reading (%)': '{:.2f}',
                        'Pass Overall (%)' : '{:.2f}'})


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Pass Math (%),Pass Reading (%),Pass Overall (%)
0,15,"39,170","$24,649,428",78.99,81.88,74.98,85.81,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [4]:
# Calculate Average Reading and Math Scores
Average_Score = df.groupby('school_name').mean()

#Keep DataFrame and remove unncessary columns
Summary_df = Average_Score.drop(columns=['Student ID', 'School ID', 'size', 'budget'])

#Merge new Data Frame with School Data Frame and remove unnecessary columns
Summary_df = pd.merge(Summary_df,  school, on ='school_name', how ='left') 
Summary_df = Summary_df.drop(columns=['School ID'])

# Calculate per student budget
Summary_df['Per Student Budget (USD)'] = Summary_df['budget']/Summary_df['size']



# Calculate total students per school who passed math, passed reading, and passed both
Pass_Math_School = df.groupby('school_name')['math_score'].apply(lambda x: (x >= 70).sum()).reset_index(name='Pass Math Count')
Pass_Read_School = df.groupby('school_name')['reading_score'].apply(lambda x: (x >= 70).sum()).reset_index(name='Pass Reading Count')
Pass_Both = df[(df['math_score'] >= 70) & (df['reading_score'] >= 70)].groupby('school_name').count()

# Clean DataFrame with both passing score
Pass_Both = pd.DataFrame(Pass_Both['student_name'])
Pass_Both.rename(columns={'student_name':'Pass_Both_Count'}, inplace=True)

# Merge Previously Created Data Frames
Merge_Summary = pd.merge(Summary_df, Pass_Math_School, on ='school_name', how ='left')
Merge_Summary = pd.merge(Merge_Summary, Pass_Read_School, on ='school_name', how ='left')
Merge_Summary = pd.merge(Merge_Summary, Pass_Both, on ='school_name', how ='left')

# Calculate Percentage of Students Who Passed in each school
Merge_Summary['Pass Math (%)'] = Merge_Summary['Pass Math Count']/Merge_Summary['size'] * 100
Merge_Summary['Pass Reading (%)'] = Merge_Summary['Pass Reading Count']/Merge_Summary['size'] * 100
Merge_Summary['Pass Overall (%)'] = Merge_Summary['Pass_Both_Count']/Merge_Summary['size'] * 100

# Drop Unecessary Columns
Merge_Summary = Merge_Summary.drop(['Pass Math Count', 'Pass Reading Count', 'Pass_Both_Count'], axis = 1) 

#Rename Columns for a more Descriptive Title
Merge_Summary = Merge_Summary.rename(columns={"school_name": "School Name", "reading_score": "Average Reading Score", 
                                             'math_score' : 'Average Math Score', 'type' : "School Type", 'size' : 'Total Students',
                                             'budget': 'Total School Budget'})

# Re-Order Columns
Merge_Summary = Merge_Summary[['School Name', 'School Type', 'Total Students', 'Total School Budget', 'Per Student Budget (USD)', 
                               'Average Math Score', 'Average Reading Score', 'Pass Math (%)', 'Pass Reading (%)',
                               'Pass Overall (%)']]

Merge_Summary = Merge_Summary.set_index('School Name')

# Format columns of Summary Data Frame for ease of reading
Merge_Summary_Formated = Merge_Summary.style.format({"Total Students": '{:,}', 'Total School Budget': '${:,.2f}', 'Average Math Score': '{:.6f}',
                        'Average Reading Score': '{:.6f}', 'Pass Math (%)': '{:.6f}', 'Pass Reading (%)': '{:.6f}',
                        'Pass Overall (%)' : '{:.6f}', 'Per Student Budget (USD)' : '${:.2f}'})

Merge_Summary_Formated

,School Type,Total Students,Total School Budget,Per Student Budget (USD),Average Math Score,Average Reading Score,Pass Math (%),Pass Reading (%),Pass Overall (%)
School Name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [5]:
# Using previously generated dataframe and sorting by overall passing rate and specifying that we want
# the results in descending order
Schools_Performance_Best = Merge_Summary.sort_values(by=['Pass Overall (%)'], ascending=False)

# Using .head() here would have worked, but I decided to call the first 5 rows instead for consistency with the next
# portion of the activity
Schools_Performance_Best = Schools_Performance_Best.iloc[:5]

# Format the DataFrame
Schools_Performance_Best = Schools_Performance_Best.style.format({"Total Students": '{:,}', 'Total School Budget': '${:,.2f}', 'Average Math Score': '{:.6f}',
                        'Average Reading Score': '{:.6f}', 'Pass Math (%)': '{:.6f}', 'Pass Reading (%)': '{:.6f}',
                        'Pass Overall (%)' : '{:.6f}', 'Per Student Budget (USD)' : '${:.2f}'})

# Call the DataFrame
Schools_Performance_Best


,School Type,Total Students,Total School Budget,Per Student Budget (USD),Average Math Score,Average Reading Score,Pass Math (%),Pass Reading (%),Pass Overall (%)
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [6]:
# Using previously generated dataframe and sorting by overall passing rate and defaulting to acending order
Schools_Performance_Worst = Merge_Summary.sort_values(by=['Pass Overall (%)'])

# Same as above example. Calling .head() would have the same effect
Schools_Performance_Worst = Schools_Performance_Worst.iloc[:5]

# Format the DataFrame
Schools_Performance_Worst = Schools_Performance_Worst.style.format({"Total Students": '{:,}', 'Total School Budget': '${:,.2f}', 'Average Math Score': '{:.6f}',
                        'Average Reading Score': '{:.6f}', 'Pass Math (%)': '{:.6f}', 'Pass Reading (%)': '{:.6f}',
                        'Pass Overall (%)' : '{:.6f}', 'Per Student Budget (USD)' : '${:.2f}'})

# Call the DataFrame
Schools_Performance_Worst

,School Type,Total Students,Total School Budget,Per Student Budget (USD),Average Math Score,Average Reading Score,Pass Math (%),Pass Reading (%),Pass Overall (%)
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [7]:
# Create a new series with all of the 9th grade results, grouping by school name and then calling the mean function
math_9th = (student.loc[student['grade']=='9th'].groupby('school_name'))['math_score'].mean()

# Create a new series with all of the 10th grade results, grouping by school name and then calling the mean function
math_10th = (student.loc[student['grade']=='10th'].groupby('school_name'))['math_score'].mean()

# Create a new series with all of the 11th grade results, grouping by school name and then calling the mean function
math_11th = (student.loc[student['grade']=='11th'].groupby('school_name'))['math_score'].mean()

# Create a new series with all of the 12th grade results, grouping by school name and then calling the mean function
math_12th = (student.loc[student['grade']=='12th'].groupby('school_name'))['math_score'].mean()

#Create new DF with the previously generated series'
math_by_grade = pd.DataFrame({'9th' : math_9th, '10th' : math_10th, '11th' : math_11th, '12th' : math_12th})

# Remove the name of the Index for cleaner formatting
math_by_grade.index.names = ['']

# Call the DataFrame
math_by_grade

## This will give us the necessary values, however it isnt in the format required
# Math_Score = student.groupby(['school_name', "grade"=="9th"]).agg({'math_score': 'mean'})

,9th,10th,11th,12th
,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:
# Create a new series with all of the 9th grade results, grouping by school name and then calling the mean function
read_9th = (student.loc[student['grade']=='9th'].groupby('school_name'))['reading_score'].mean()

# Create a new series with all of the 10th grade results, grouping by school name and then calling the mean function
read_10th = (student.loc[student['grade']=='10th'].groupby('school_name'))['reading_score'].mean()

# Create a new series with all of the 11th grade results, grouping by school name and then calling the mean function
read_11th = (student.loc[student['grade']=='11th'].groupby('school_name'))['reading_score'].mean()

# Create a new series with all of the 12th grade results, grouping by school name and then calling the mean function
read_12th = (student.loc[student['grade']=='12th'].groupby('school_name'))['reading_score'].mean()

#Create new DF with the previously generated series'
read_by_grade = pd.DataFrame({'9th' : read_9th, '10th' : read_10th, '11th' : read_11th, '12th' : read_12th})

# Remove the name of the Index for cleaner formatting
read_by_grade.index.names = ['']

# Call the DataFrame
read_by_grade

,9th,10th,11th,12th
,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [9]:
# Create desired bins as well as their respective names
bins = [575, 594.9, 614.9, 634.9, 655]
group_names = ['575 - 595', '595 - 615', '615 - 635', '635 - 655']

# Copy previous df for merging and editing
Merge_Summary_Spend = Merge_Summary

#Cut and Bin the data
Merge_Summary_Spend['Spending Ranges ($ Per Student)'] = pd.cut(Merge_Summary_Spend['Per Student Budget (USD)'],
                                                                bins, labels = group_names)

# Create a new series with all of the math score results, grouping by avg school spending and then calling 
# the mean function
Avg_math = (Merge_Summary_Spend.groupby('Spending Ranges ($ Per Student)'))['Average Math Score'].mean()

# Create a new series with all of the reading score results, grouping by avg school spending and then calling 
# the mean function
Avg_read = (Merge_Summary_Spend.groupby('Spending Ranges ($ Per Student)'))['Average Reading Score'].mean()

# Create a new series with all of the math pass percentage results, grouping by avg school spending and then calling 
# the mean function
per_pass_math = (Merge_Summary_Spend.groupby('Spending Ranges ($ Per Student)'))['Pass Math (%)'].mean()

# Create a new series with all of the reading pass percentage results, grouping by avg school spending and then calling 
# the mean function
per_pass_read = (Merge_Summary_Spend.groupby('Spending Ranges ($ Per Student)'))['Pass Reading (%)'].mean()

# Finding the overall passing rate by adding the two values and dividing by 2
overall_pass = (per_pass_math + per_pass_read) / 2

#Create new DF with the previously generated series'
Scores_Spend = pd.DataFrame({'Average Math Score' : Avg_math, 'Average Reading Score' : Avg_read,
                             '% Passing Math' : per_pass_math, '% Passing Reading' : per_pass_read,
                             '% Overall Passing' : overall_pass})

# Format the Series
Scores_Spend_Format = Scores_Spend.style.format({"Average Math Score": '{:.2f}', 
                                                 'Average Reading Score': '{:.2f}', 
                                                 '% Passing Math': '{:.2f}',
                                                 '% Passing Reading': '{:.2f}', 
                                                 '% Overall Passing': '{:.2f}'})

# Call the df
Scores_Spend_Format

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges ($ Per Student),,,,,
575 - 595,83.46,83.93,93.46,96.61,95.04
595 - 615,83.60,83.89,94.23,95.90,95.07
615 - 635,80.20,82.43,80.04,89.54,84.79
635 - 655,77.87,81.37,70.35,83.00,76.67


## Scores by School Size

* Perform the same operations as above, based on school size.

In [10]:
# Create desired bins as well as their respective names
Size_bins = [0, 1249, 2499, 3749, 5000]
Size_names = ['Small (< 1250)', 'Medium (1250 - 2500)', 'Large (2500 - 3750)', 'Extra Large (3750 - 5000)']

# Copy previous df for merging and editing
Merge_Summary_Size = Merge_Summary

#Cut and Bin the data
Merge_Summary_Size['School Population Range'] = pd.cut(Merge_Summary_Size['Total Students'], Size_bins, labels = Size_names)

# Create a new series with all of the math score results, grouping by avg school size and then calling 
# the mean function
Avg_math = (Merge_Summary_Size.groupby('School Population Range'))['Average Math Score'].mean()

# Create a new series with all of the reading score results, grouping by avg school size and then calling 
# the mean function
Avg_read = (Merge_Summary_Size.groupby('School Population Range'))['Average Reading Score'].mean()

# Create a new series with all of the math pass percentage results, grouping by avg school size and then calling 
# the mean function
per_pass_math = (Merge_Summary_Size.groupby('School Population Range'))['Pass Math (%)'].mean()

# Create a new series with all of the reading pass percentage results, grouping by avg school size and then calling 
# the mean function
per_pass_read = (Merge_Summary_Size.groupby('School Population Range'))['Pass Reading (%)'].mean()

# Finding the overall passing rate by adding the two values and dividing by 2
overall_pass = (per_pass_math + per_pass_read) / 2

#Create new DF with the previously generated series'
Scores_Size = pd.DataFrame({'Average Math Score' : Avg_math, 'Average Reading Score' : Avg_read,
                             '% Passing Math' : per_pass_math, '% Passing Reading' : per_pass_read,
                             '% Overall Passing' : overall_pass})

# Format the DataFrame
Scores_Size_Format = Scores_Size.style.format({"Average Math Score": '{:.2f}', 
                                                 'Average Reading Score': '{:.2f}', 
                                                 '% Passing Math': '{:.2f}',
                                                 '% Passing Reading': '{:.2f}', 
                                                 '% Overall Passing': '{:.2f}'})

# Call the df
Scores_Size_Format

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Population Range,,,,,
Small (< 1250),83.82,83.93,93.55,96.10,94.82
Medium (1250 - 2500),83.36,83.89,93.64,96.75,95.20
Large (2500 - 3750),76.81,81.03,66.66,80.45,73.56
Extra Large (3750 - 5000),77.06,80.92,66.46,81.06,73.76


## Scores by School Type

* Perform the same operations as above, based on school type

In [11]:
# Copy previous df for merging and editing
Merge_Summary_Type = Merge_Summary

# Create a new series with all of the math score results, grouping by school type and then calling 
# the mean function
Avg_math = (Merge_Summary_Type.groupby('School Type'))['Average Math Score'].mean()

# Create a new series with all of the reading score results, grouping by school type and then calling 
# the mean function
Avg_read = (Merge_Summary_Type.groupby('School Type'))['Average Reading Score'].mean()

# Create a new series with all of the math pass percentage results, grouping by school type and then calling 
# the mean function
per_pass_math = (Merge_Summary_Type.groupby('School Type'))['Pass Math (%)'].mean()

# Create a new series with all of the reading pass percentage results, grouping by school type and then calling 
# the mean function
per_pass_read = (Merge_Summary_Type.groupby('School Type'))['Pass Reading (%)'].mean()

# Finding the overall passing rate by adding the two values and dividing by 2
overall_pass = (per_pass_math + per_pass_read) / 2

#Create new DF with the previously generated series'
Scores_Type = pd.DataFrame({'Average Math Score' : Avg_math, 'Average Reading Score' : Avg_read,
                             '% Passing Math' : per_pass_math, '% Passing Reading' : per_pass_read,
                             '% Overall Passing' : overall_pass})

# Format the DataFrame
Scores_Type_Format = Scores_Type.style.format({"Average Math Score": '{:.2f}', 
                                                 'Average Reading Score': '{:.2f}', 
                                                 '% Passing Math': '{:.2f}',
                                                 '% Passing Reading': '{:.2f}', 
                                                 '% Overall Passing': '{:.2f}'})
# Call the df
Scores_Type_Format

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,95.10
District,76.96,80.97,66.55,80.80,73.67
